<a href="https://colab.research.google.com/github/Yashwithareddy01/yashwitha/blob/main/vinayaka_shortdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357273 sha256=4be57a4b611397ec28059c89c23be81489e217b5ea7bef7a96a85facf327bd9e
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 48.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import train_test_split as sklearn_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from implicit.als import AlternatingLeastSquares
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 1: Load and Prepare Data
movies = pd.read_csv('/content/drive/MyDrive/Hybrid-Recommendation-System-master/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Hybrid-Recommendation-System-master/ratings.csv')

# Data cleaning
movies.dropna(inplace=True)
ratings.dropna(inplace=True)

# Step 2: Normalize ratings (0.5 to 5)
scaler = MinMaxScaler(feature_range=(0.5, 5))
ratings['rating'] = scaler.fit_transform(ratings[['rating']])

# Step 3: Split data into x_train, y_train, x_test, and y_test
x = ratings[['userId', 'movieId']]  # Features (UserID and MovieID)
y = ratings['rating']                # Target (Ratings)

# Split into training and testing set (80% train, 20% test)
x_train, x_test, y_train, y_test = sklearn_split(x, y, test_size=0.2, random_state=42)

# Step 4: Prepare the Surprise Dataset for SVD
reader = Reader(rating_scale=(0.5, 5.0))
train_data = Dataset.load_from_df(pd.concat([x_train, y_train], axis=1), reader)
trainset = train_data.build_full_trainset()

testset = [(row['userId'], row['movieId'], y_true) for (row, y_true) in zip(x_test.to_dict('records'), y_test)]

In [ ]:
 #Step 5: Hyperparameter tuning for SVD model
param_grid = {
    'n_factors': [20, 50, 100],        # Number of latent factors
    'reg_all': [0.02, 0.1, 0.2],       # Regularization term
    'lr_all': [0.002, 0.005, 0.01]     # Learning rate
}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
grid_search.fit(train_data)

In [ ]:



#

# Best SVD model after hyperparameter tuning
best_svd_model = grid_search.best_estimator['rmse']
print("Best SVD model parameters:", grid_search.best_params['rmse'])

# Step 6: Train the best SVD model on training data
best_svd_model.fit(trainset)

# Step 7: Predictions using the best SVD model
svd_predictions = best_svd_model.test(testset)

# Step 8: Evaluate the SVD model
rmse_svd = accuracy.rmse(svd_predictions)
mae_svd = accuracy.mae(svd_predictions)

# Step 9: Evaluate precision@k and recall@k for SVD
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    user_est_true = {}
    for pred in predictions:
        user_est_true.setdefault(pred.uid, []).append((pred.est, pred.r_ui))

    precisions = []
    recalls = []
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        top_k_ratings = user_ratings[:k]

        n_rel = sum((true_r > threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est_r > threshold) for (est_r, _) in top_k_ratings)
        n_rel_and_rec_k = sum(((true_r > threshold) and (est_r > threshold))
                              for (est_r, true_r) in top_k_ratings)

        precisions.append(n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1)
        recalls.append(n_rel_and_rec_k / n_rel if n_rel != 0 else 1)

    return np.mean(precisions), np.mean(recalls)

precision_svd, recall_svd = precision_recall_at_k(svd_predictions, k=10)



Best SVD model parameters: {'n_factors': 100, 'reg_all': 0.1, 'lr_all': 0.01}
RMSE: 0.8722
MAE:  0.6679


In [ ]:
# Step 10: Alternating Least Squares (ALS) model
# Prepare data for ALS (create a sparse matrix)
user_item_matrix = coo_matrix((ratings['rating'], (ratings['userId'], ratings['movieId'])))
als_model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=10)
als_model.fit(user_item_matrix.T)

user_id = 1  # Example user ID

# Create a user_items matrix with a single row for the target user
user_items = user_item_matrix.tocsr()[user_id - 1, :].tocsr()  # Adjust for zero-based indexing

# Get recommendations for the target user
als_user_recs = als_model.recommend(user_id, user_items, N=10)
print("ALS Recommendations for user 1:", als_user_recs)

# Step 12: Evaluation for ALS
# Get predicted ratings for ALS
# Here we ensure we use the correct user_item_matrix for predictions
#als_user_ratings = user_item_matrix.dot(als_model.item_factors.T) # Incorrect: Dimension mismatch

# Corrected line: Multiply user factors by item factors
als_user_ratings = als_model.user_factors.dot(als_model.item_factors.T)

# Debugging prints to check dimensions
print("Shape of user_item_matrix:", user_item_matrix.shape)
print("Shape of als_model.item_factors:", als_model.item_factors.shape)
print("Shape of als_model.user_factors:", als_model.user_factors.shape) # Add this line

# Check if the multiplication can happen
try:
    als_rmse = sqrt(mean_squared_error(user_item_matrix.toarray().ravel(), als_user_ratings.ravel()))
    als_mae = mean_absolute_error(user_item_matrix.toarray().ravel(), als_user_ratings.ravel())
    print(f"ALS RMSE: {als_rmse}")
    print(f"ALS MAE: {als_mae}")
except ValueError as e:
    print(f"Error during evaluation: {e}")

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0030367374420166016 seconds
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

ALS Recommendations for user 1: (array([534, 514, 202, 111, 477,   6, 381,  45, 462,  84], dtype=int32), array([1.3590391, 1.3263662, 1.3240246, 1.3199035, 1.3165274, 1.2957572,
       1.2849975, 1.2775058, 1.2617396, 1.2360325], dtype=float32))
Shape of user_item_matrix: (611, 193610)
Shape of als_model.item_factors: (611, 50)
Shape of als_model.user_factors: (193610, 50)
ALS RMSE: 0.11067418944897683
ALS MAE: 0.005720024089396643


In [ ]:
# Step 13: Cosine Similarity
# Create user-item matrix for cosine similarity
user_item_matrix_full = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Calculate cosine similarity between movies
movie_similarity = cosine_similarity(user_item_matrix_full.T)

# Function to get movie recommendations based on cosine similarity

In [ ]:
def get_movie_recommendations(movie_id, movie_similarity, movies, top_n=10):
    movie_idx = movies[movies['movieId'] == movie_id].index[0]
    similar_indices = movie_similarity[movie_idx].argsort()[-top_n-1:-1][::-1]  # Get indices of top N similar movies
    return movies.iloc[similar_indices]

# Example: Get recommendations for a specific movie (replace movie_id with your choice)
movie_id_example = 1  # Replace with a movieId you want recommendations for
recommended_movies = get_movie_recommendations(movie_id_example, movie_similarity, movies)
print(f"Movies recommended based on movie ID {movie_id_example}:\n", recommended_movies)

# Step 14: Print out all metrics
print(f"SVD RMSE: {rmse_svd}")
print(f"SVD MAE: {mae_svd}")
print(f"SVD Precision@10: {precision_svd}")
print(f"SVD Recall@10: {recall_svd}")
print(f"ALS RMSE: {als_rmse}")
print(f"ALS MAE: {als_mae}")

# Step 15: Print ALS recommendations
als_movie_ids = [item[0] for item in als_user_recs]
als_movie_titles = movies[movies['movieId'].isin(als_movie_ids)]
print("ALS Recommendations (Movie Titles):\n", als_movie_titles)


Movies recommended based on movie ID 1:
       movieId                                              title  \
2353     3112                               'night Mother (1986)   
418       480                               Jurassic Park (1993)   
615       780               Independence Day (a.k.a. ID4) (1996)   
224       260          Star Wars: Episode IV - A New Hope (1977)   
314       356                                Forrest Gump (1994)   
322       364                              Lion King, The (1994)   
910      1209  Once Upon a Time in the West (C'era una volta ...   
546       648                         Mission: Impossible (1996)   
963      1264                                        Diva (1981)   
968      1269                        Arsenic and Old Lace (1944)   

                                               genres  
2353                                            Drama  
418                  Action|Adventure|Sci-Fi|Thriller  
615                  Action|Adventure|Sci-